# Banking Customer & Investment Analysis

This notebook explores customer behaviour, transaction patterns, and investment dynamics 
within a retail banking context.


### **0. Import Librairies**

In [ ]:
import csv
import pandas as pd
import seaborn as sns
import numpy as np 
import matplotlib.pyplot as plt


#for part 7:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import (
    classification_report, confusion_matrix, accuracy_score,
    mean_squared_error, r2_score
)

#for part 8
import os 

### **1. Import data**

In [ ]:
#create a function 
def read_semicolon_csv(path):
    df = pd.read_csv(
        path,
        sep=";",
        engine="python",
        quoting=csv.QUOTE_NONE
    )
    #clean quotes 
    df.columns = df.columns.str.replace('"', '', regex=False)
    return df

#apply the function 
df_customer = read_semicolon_csv(
    "your_path_to_datasets/customer_features.csv"
)

df_branch = read_semicolon_csv(
    "your_path_to_datasets/branch_features.csv"
)

df_transaction = read_semicolon_csv(
    "your_path_to_datasets/transaction_features.csv"
)



1.1 Verification of importations 

In [ ]:
# verification
print(df_customer.shape, df_customer.columns.tolist())
print(df_branch.shape, df_branch.columns.tolist())
print(df_transaction.shape, df_transaction.columns.tolist())


### **2. Temporal Variables & Coverage Validation**

2.1 Quick checks

In [ ]:
#dtypes
print(df_transaction[["txn_year", "txn_month"]].dtypes)

#ensure year and month are numeric
df_transaction["txn_year"] = pd.to_numeric(df_transaction["txn_year"], errors="coerce")
df_transaction["txn_month"] = pd.to_numeric(df_transaction["txn_month"], errors="coerce")

2.2 Create a synthetic date / period

In [ ]:
df_transaction["year_month"] = (
    df_transaction["txn_year"].astype(str) + "-" + 
    df_transaction["txn_month"].astype(str).str.zfill(2)
)
#convert to pandas Period format (YYYY-MM)
df_transaction["year_month"] = pd.PeriodIndex(df_transaction["year_month"], freq="M")

2.3 Coverage check

In [ ]:
print("Time coverage:")
print("Earliest period:", df_transaction["year_month"].min())
print("Latest period:  ", df_transaction["year_month"].max())

2.4. Count how many transactions per period

In [ ]:
period_counts = df_transaction["year_month"].value_counts().sort_index()
print("\nTransactions per period:")
print(period_counts)

2.5 Identify missing months (if any) 

In [ ]:
full_range = pd.period_range(
    start=df_transaction["year_month"].min(),
    end=df_transaction["year_month"].max(),
    freq="M"
)

missing_months = full_range.difference(df_transaction["year_month"].unique())
print("\nMissing months:", missing_months)

### **3. Customer EDA**

In [ ]:
#Quick overview
print(df_customer.head())
print(df_customer.describe(include='all'))

3.1 Age distribution

In [ ]:
print(list(df_customer.columns))


In [ ]:
plt.figure(figsize=(8,5))
sns.histplot(df_customer['Age'], bins=20, kde=True)
plt.title("Customer Age Distribution")
plt.xlabel("Age")
plt.ylabel("Count")
plt.show()

3.2 Customer Type breakdown

In [ ]:
plt.figure(figsize=(6,4))
sns.countplot(data=df_customer, x="Customer_Type")
plt.title("Customer Type Distribution")
plt.show()

# Value counts
print("\nCustomer Type counts:")
print(df_customer['Customer_Type'].value_counts())

3.3 Region distribution

In [ ]:
plt.figure(figsize=(8,4))
sns.countplot(data=df_customer, x="customer_region")
plt.title("Customer Region Distribution")
plt.show()

print("\nRegion counts:")
print(df_customer['customer_region'].value_counts())

3.4 Average Balance distribution

In [ ]:
plt.figure(figsize=(8,5))
sns.histplot(df_customer['avg_balance'], bins=30, kde=True)
plt.title("Average Customer Balance Distribution")
plt.xlabel("Average Balance")
plt.ylabel("Count")
plt.show()

3.5 Balance vs Investment Status

In [ ]:
plt.figure(figsize=(6,4))
sns.boxplot(data=df_customer, x='is_investor', y='avg_balance')
plt.title("Average Balance by Investor Status")
plt.xlabel("Is Investor")
plt.ylabel("Average Balance")
plt.show()

print("\nAverage balance by investor status:")
print(df_customer.groupby('is_investor')['avg_balance'].mean())

### **4.Transaction EDA**

In [ ]:
print(df_transaction.head())
print(df_transaction.describe(include='all'))

4.1 Distribution of Transaction Amounts

In [ ]:
plt.figure(figsize=(8,5))
sns.histplot(df_transaction['Transaction_Amount'], bins=40, kde=True)
plt.title("Transaction Amount Distribution")
plt.xlabel("Transaction Amount")
plt.ylabel("Count")
plt.show()

4.2 Distribution of Investment Amounts

In [ ]:
plt.figure(figsize=(8,5))
sns.histplot(df_transaction['Investment_Amount'], bins=40, kde=True)
plt.title("Investment Amount Distribution")
plt.xlabel("Investment Amount")
plt.ylabel("Count")
plt.show()

print(df_transaction["Investment_Amount"].describe())

4.3 Investment Type Breakdown

In [ ]:
plt.figure(figsize=(6,4))
sns.countplot(data=df_transaction, x="Investment_Type")
plt.title("Investment Type Distribution")
plt.xticks(rotation=45)
plt.show()

print(df_transaction["Investment_Type"].value_counts())

4.4 Monthly Transaction Volume

In [ ]:
tx_by_month = df_transaction.groupby("year_month")["Transaction_ID"].count()

plt.figure(figsize=(10,5))
tx_by_month.plot(kind="line", marker="o")
plt.title("Monthly Transaction Volume Over Time")
plt.xlabel("Year-Month")
plt.ylabel("Number of Transactions")
plt.grid(True)
plt.show()

4.5 Average Transaction Amount per Month

In [ ]:
avg_tx_month = df_transaction.groupby("year_month")["Transaction_Amount"].mean()

plt.figure(figsize=(10,5))
avg_tx_month.plot(kind="line", marker="o", color="orange")
plt.title("Average Transaction Amount per Month")
plt.xlabel("Year-Month")
plt.ylabel("Avg Transaction Amount")
plt.grid(True)
plt.show()

### **5. Correlation Analysis**

5.1 select relevant numeric columns

In [ ]:
numeric_cols = [
    "Age",
    "avg_balance",
    "avg_tx_amount",
    "avg_investment",
    "num_transactions",
    "num_investment_operations"
]

df_numeric = df_customer[numeric_cols]
df_numeric.head()

5.2 Correlation matrix

In [ ]:
corr_matrix = df_numeric.corr()
print(corr_matrix)

5.3 Heatmap of correlations

In [ ]:
sns.heatmap(corr_matrix, annot=True, cmap="Blues", fmt=".2f")
plt.title("Correlation Matrix of Customer Features")
plt.show()

5.4 Correlation with investment-related features

In [ ]:
investment_related = df_customer[["avg_balance", "avg_investment", "num_investment_operations"]]
print(investment_related.corr())

### **6. Probability Analysis**

In [ ]:
df_prob = df_customer.copy()

# Create age groups
df_prob["age_group"] = pd.cut(
    df_prob["Age"],
    bins=[18, 30, 45, 60, 80],
    labels=["18-30", "31-45", "46-60", "61-80"]
)

# Define high investors as top 30% investment amount
threshold_invest = df_prob["avg_investment"].quantile(0.70)
df_prob["is_high_investor"] = (df_prob["avg_investment"] >= threshold_invest).astype(int)

# High-frequency investors = top 30% num_investment_operations
threshold_freq = df_prob["num_investment_operations"].quantile(0.70)
df_prob["is_high_freq"] = (df_prob["num_investment_operations"] >= threshold_freq).astype(int)

# High balance = top 30%
threshold_balance = df_prob["avg_balance"].quantile(0.70)
df_prob["is_high_balance"] = (df_prob["avg_balance"] >= threshold_balance).astype(int)

df_prob.head()

6.1 Unconditional probability

In [ ]:
p_high_invest = df_prob["is_high_investor"].mean()
print("P(high investment):", round(p_high_invest, 3))

6.2 P(high investment | age_group)

In [ ]:
p_age = df_prob.groupby("age_group")["is_high_investor"].mean()
print("\nP(high investment | age group):\n", p_age)

6.3 P(high investment | region)

In [ ]:
p_region = df_prob.groupby("customer_region")["is_high_investor"].mean()
print("\nP(high investment | region):\n", p_region)

6.4 P(high investment | customer type)

In [ ]:
p_type = df_prob.groupby("Customer_Type")["is_high_investor"].mean()
print("\nP(high investment | customer type):\n", p_type)

6.5 P(high investment | high balance)

In [ ]:
p_balance = df_prob.groupby("is_high_balance")["is_high_investor"].mean()
print("\nP(high investment | high balance):\n", p_balance)

### **7. Predictive Analysis**

### Models:
- (1) Logistic Regression → Predict high-value investors
- (2) Linear Regression  → Predict average investment amount

In [ ]:
df_model = df_prob.copy()

# Feature lists
cat_features = ["Customer_Type", "customer_region"]
num_features = [
    "Age", 
    "avg_balance", 
    "avg_tx_amount", 
    "num_transactions",
    "num_investment_operations"
]

# Preprocessing object
preprocess = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(drop="first"), cat_features),
        ("num", "passthrough", num_features)
    ]
)

7.1 Logistic Regression - Classification

In [ ]:
X_class = df_model[cat_features + num_features]
y_class = df_model["is_high_investor"]

X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(
    X_class, y_class, test_size=0.25, random_state=42, stratify=y_class
)

7.2 Build logisitic regression pipeline

In [ ]:
log_reg_model = Pipeline(
    steps=[
        ("preprocess", preprocess),
        ("classifier", LogisticRegression(max_iter=1000))
    ]
)

# Train
log_reg_model.fit(X_train_c, y_train_c)

# Predict
y_pred_c = log_reg_model.predict(X_test_c)

7.3 Evaluate logistic regresison 

In [ ]:
print("=== Logistic Regression Evaluation ===")
print("Accuracy:", round(accuracy_score(y_test_c, y_pred_c), 3))

print("\nClassification Report:\n", classification_report(y_test_c, y_pred_c))

print("Confusion Matrix:")
print(confusion_matrix(y_test_c, y_pred_c))

7.4 intepret model coefficient (importance)

In [ ]:
# Extract feature names from the preprocessing step
ohe = log_reg_model.named_steps["preprocess"].transformers_[0][1]
encoded_cat_names = ohe.get_feature_names_out(cat_features).tolist()
all_feature_names = encoded_cat_names + num_features

# Extract coefficients
coefs = log_reg_model.named_steps["classifier"].coef_[0]
importance = sorted(zip(coefs, all_feature_names), reverse=True)

print("\n=== Logistic Regression Feature Importance ===")
for coef, name in importance:
    print(f"{name}: {coef:.4f}")

7.5 Linear Regression — Continuous Prediction

In [ ]:
X_reg = df_model[cat_features + num_features]
y_reg = df_model["avg_investment"]

X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(
    X_reg, y_reg, test_size=0.25, random_state=42
)


#Build linear regression pipeline
lin_reg_model = Pipeline(
    steps=[
        ("preprocess", preprocess),
        ("regressor", LinearRegression())
    ]
)

# Train
lin_reg_model.fit(X_train_r, y_train_r)

# Predict
y_pred_r = lin_reg_model.predict(X_test_r)

#evaluate
print("\n=== Linear Regression Evaluation ===")
print("MSE:", round(mean_squared_error(y_test_r, y_pred_r), 2))
print("R² Score:", round(r2_score(y_test_r, y_pred_r), 3))

#interprete
# Extract regression coefficients
reg_coefs = lin_reg_model.named_steps["regressor"].coef_
importance_reg = sorted(zip(reg_coefs, all_feature_names), reverse=True)

print("\n=== Linear Regression Coefficients ===")
for coef, name in importance_reg:
    print(f"{name}: {coef:.2f}")

### **8. Export dashboard datasets for Tableau**

In [ ]:
# ============================
# Export predictions
# ============================

output_path = "your_path"
os.makedirs(output_path, exist_ok=True)

THRESHOLD = 0.5

df_pred = df_model.copy()

# Safety check: indices should match
assert df_pred.index.equals(df_model.index)

# Train/test split flag
df_pred["split"] = "train"
df_pred.loc[X_test_c.index, "split"] = "test"

# Store target for dashboard evaluation
df_pred["target_is_high_investor"] = y_class

# Logistic regression probability + class
df_pred["pred_proba_high_investor"] = log_reg_model.predict_proba(X_class)[:, 1]
df_pred["pred_class_high_investor"] = (df_pred["pred_proba_high_investor"] >= THRESHOLD).astype(int)

# Linear regression prediction (continuous)
df_pred["pred_avg_investment"] = lin_reg_model.predict(X_reg)

# Probability deciles (for lift charts / segmentation)
df_pred["proba_decile"] = pd.qcut(
    df_pred["pred_proba_high_investor"],
    q=10,
    labels=False,
    duplicates="drop"
) + 1

pred_file = os.path.join(output_path, "dashboard_predictions.csv")
df_pred.to_csv(pred_file, index=False, encoding="utf-8")
print(f"Predictions dataset exported → {pred_file}")


# ============================
# Export model coefficients
# ============================

df_importance_log = pd.DataFrame({
    "feature": all_feature_names,
    "logistic_coef": coefs
}).sort_values("logistic_coef", ascending=False)

df_importance_lin = pd.DataFrame({
    "feature": all_feature_names,
    "linear_coef": reg_coefs
}).sort_values("linear_coef", ascending=False)

importance_log_file = os.path.join(output_path, "model_importance_logistic.csv")
importance_lin_file = os.path.join(output_path, "model_importance_linear.csv")

df_importance_log.to_csv(importance_log_file, index=False, encoding="utf-8")
df_importance_lin.to_csv(importance_lin_file, index=False, encoding="utf-8")

print(f"Logistic importance exported → {importance_log_file}")
print(f"Linear importance exported → {importance_lin_file}")


# ============================
# Export datasets
# ============================

customer_file = os.path.join(output_path, "dashboard_customer.csv")
df_prob.to_csv(customer_file, index=False, encoding="utf-8")
print(f"Customer dataset exported → {customer_file}")

transaction_file = os.path.join(output_path, "dashboard_transaction.csv")
df_transaction_dashboard = df_transaction.copy()
df_transaction_dashboard["year_month"] = df_transaction_dashboard["year_month"].astype(str)
df_transaction_dashboard.to_csv(transaction_file, index=False, encoding="utf-8")
print(f"Transaction dataset exported → {transaction_file}")

branch_file = os.path.join(output_path, "dashboard_branch.csv")
df_branch.to_csv(branch_file, index=False, encoding="utf-8")
print(f"Branch dataset exported → {branch_file}")
